In [1]:
!pip install transformers
!pip install torch

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip


Using a Zero-shot Classifier from facebook

In [12]:
import pandas as pd
import json
import re
from transformers import pipeline
import torch
from tqdm import tqdm

def clean_address(address):
    """Clean and preprocess address text"""
    if pd.isna(address):
        return ""
        
    # Rule1: remove urls
    address = re.sub(r'(https?://\S+)', "", address)
    # Rule2: replace "escape chars" by "space"
    address = re.sub(r"[\n\t\r]", " ", address)
    # Rule3: replace 'apostrophe s' by 's'
    address = re.sub(r"[\'\"\`]s", "s", address)
    # Rule4: remove single/double quotes having space on either side
    address = re.sub(r"[\'\"] ", " ", address)
    address = re.sub(r" [\'\"]", " ", address)
    # Rule5: replace "single/double quotes surrounded by multiple alphabets" by "space"
    address = re.sub(r"[a-zA-Z]{2,}[\'\"][a-zA-Z]{2,}", " ", address)
    # Rule6: replace '=', ':', '~' by '-'
    address = re.sub(r"[\=\:\~]", "-", address)
    # Rule7: replace square/curly brackets by round brackets
    address = re.sub(r"[\[\{]", "(", address)
    address = re.sub(r"[\]\}]", ")", address)
    # Rule8: replace 'pipe' and 'backslash' by '/'
    address = re.sub(r"[\|\\]", "/", address)
    # Rule9: replace ';' and '?' by ','
    address = re.sub(r"[;\?]", ",", address)
    # Rule10: replace '` ! $ @ * % < > _ ^' by space
    address = re.sub(r"[`\!\$@\*%\<\>_\^]", " ", address)
    # Rule11: collapse repeated special chars
    address = re.sub(r",+", ",", address)
    address = re.sub(r"\.+", ".", address)
    address = re.sub(r"\++", "+", address)
    address = re.sub(r"\-+", "-", address)
    address = re.sub(r"\(+", "(", address)
    address = re.sub(r"\)+", ")", address)
    address = re.sub(r"\&+", "&", address)
    address = re.sub(r"\#+", "#", address)
    address = re.sub(r"\/+", "/", address)
    address = re.sub(r"\"+", '"', address)
    address = re.sub(r"\'+", "'", address)
    address = re.sub(r" +", " ", address)
    # Rule12: strip special chars at ends
    address = address.strip()
    address = re.sub(r"^[\.\,\-\+\/\)]", "", address)
    address = re.sub(r"[\.\,\-\+\/\(]$", "", address)
    address = address.strip()
    # Rule13: replace special chars from end of tokens
    address_ = []
    for add_string in address.split():
        match_ = re.search(r"[\\.,;:\-_]+$", add_string)
        if match_:
            add_string = re.sub(r"[\\.,;:\-_]+$", " " + match_.group(0), add_string)
        address_.append(add_string)
    address = ' '.join(address_)
    address = address.lower()
    return address

def detect_repeated_words(address, entities):
    """Add unassigned label to repeated words/phrases"""
    words = {}
    for entity in entities:
        text = entity[2]
        if text in words:
            # This text appeared before, mark as unassigned
            entity[3] = "unassigned"
        else:
            words[text] = True
    return entities

def extract_meaningful_segments(address):
    """Extract meaningful segments from address using regex patterns"""
    segments = []
    
    # Define patterns for various entity types
    patterns = [
        # Unit patterns
        r'\b(?:house|flat|shop|plot|pno|h\.no|no\.)[ .]?(?:no)?[ .]?[-:]?[ ]?\d+[-\/]?\w*\b',  # house no 15, flat no 112
        r'\b\d+(?:st|nd|rd|th)[ ]\w+\b',  # 2nd floor, 5th cross
        r'\b[A-Za-z][-\/]?\d+[-\/]?\w*\b',  # B-101, C/14
        r'\b\d+[-\/]\d+(?:[-\/]\d+)*\b',  # 14-1-211/621/816
        
        # Street patterns
        r'\b\w+[ ]?(?:road|street|lane|marg|path|nagar|gali|cross|naka)\b',
        r'\b(?:old )?route (?:no )?nh[ -]?\d+\b',
        
        # Area patterns
        r'\b\w+[ ]?(?:colony|sector|block|enclave|complex|layout)\b',
        
        # Landmark patterns
        r'(?:near|opposite|behind|in ?front ?of|opp)[ ]\w+(?:[ ]\w+){0,5}',
        
        # Pincode pattern
        r'\b\d{6}\b',
        
        # Additional patterns for common Indian address components
        r'\b\w+[ ]?(?:chowk|bazaar|market|vihar|puram|society|apartment|residency)\b',
        
        # For remaining words not caught by above patterns
        r'\b\w+(?:[ ]\w+){0,2}\b'
    ]
    
    # Extract segments using patterns
    for pattern in patterns:
        for match in re.finditer(pattern, address, re.IGNORECASE):
            segment = match.group(0).strip()
            start, end = match.span()
            segments.append((segment, start, end))
    
    # Sort by position in address
    segments.sort(key=lambda x: x[1])
    
    # Handle overlaps by keeping the longest segment
    non_overlapping = []
    for segment, start, end in segments:
        add_segment = True
        for i, (existing_segment, existing_start, existing_end) in enumerate(non_overlapping):
            # Check for overlap
            if (start < existing_end and end > existing_start):
                # Keep the longer segment
                if (end - start) > (existing_end - existing_start):
                    non_overlapping[i] = (segment, start, end)
                add_segment = False
                break
        
        if add_segment:
            non_overlapping.append((segment, start, end))
    
    # Sort again by position
    non_overlapping.sort(key=lambda x: x[1])
    return non_overlapping

def is_pincode(text):
    """Check if text is a 6-digit pincode"""
    return bool(re.match(r'^\d{6}$', text))

def zero_shot_ner(addresses, batch_size=10):
    """Annotate addresses using zero-shot classification"""
    print("Loading zero-shot classification model...")
    # Load zero-shot NER model
    classifier = pipeline(
        "zero-shot-classification",
        model="facebook/bart-large-mnli",
        device=0 if torch.cuda.is_available() else -1
    )
    
    # Entity types
    entity_types = ["unit", "street_name", "society_name", "area_name", "city_name", 
                    "state_name", "landmark"]
    
    # Adding helpful context for each entity type to improve classification
    entity_context = {
        "unit": "building number, flat number, house number, floor number, shop number",
        "street_name": "road, street, lane, path, marg",
        "society_name": "apartment complex, housing society, colony, residence",
        "area_name": "neighborhood, locality, sector, district, village",
        "city_name": "city, town, municipal area",
        "state_name": "state, province, union territory",
        "landmark": "near, opposite, behind, in front of"
    }
    
    # Create a dictionary to map states and cities for additional verification
    # (You can load these from your original CSV)
    states = ["delhi", "haryana", "punjab", "maharashtra", "gujarat", "karnataka", 
              "tamil nadu", "telangana", "west bengal", "uttar pradesh", 
              "madhya pradesh", "rajasthan", "bihar", "jharkhand", "odisha"]
    
    print(f"Processing {len(addresses)} addresses in batches of {batch_size}")
    
    # Process addresses in batches to better track progress
    for batch_idx in tqdm(range(0, len(addresses), batch_size)):
        batch = addresses[batch_idx:batch_idx + batch_size]
        
        for address_data in batch:
            address = address_data["text"]
            entities = []
            
            # Skip very short addresses
            if len(address) < 5:
                address_data["entities"] = []
                continue
            
            # Extract meaningful segments from address
            segments = extract_meaningful_segments(address)
            
            for segment, start, end in segments:
                # Special case for pincodes
                if is_pincode(segment):
                    entities.append([start, end, segment, "area_pincode"])
                    continue
                
                # Special case for state names
                if segment.lower() in states:
                    entities.append([start, end, segment, "state_name"])
                    continue
                
                # Skip very short segments or common stopwords
                if len(segment) < 2 or segment.lower() in ["the", "and", "of", "to", "in", "on", "at"]:
                    continue
                
                try:
                    # Create a hypothesis for each entity type
                    hypotheses = [f"This is a {entity_type}: {entity_context[entity_type]}" 
                                 for entity_type in entity_types]
                    
                    # Classify segment
                    result = classifier(segment, hypotheses, multi_label=False)
                    
                    # Extract entity type from the result
                    top_label = result["labels"][0].split(":")[0].strip()
                    # Remove "This is a " from the beginning
                    top_label = top_label.replace("This is a ", "")
                    confidence = result["scores"][0]
                    
                    # Only add if confidence is high enough
                    if confidence > 0.65:
                        entities.append([start, end, segment, top_label])
                    else:
                        entities.append([start, end, segment, "unassigned"])
                except Exception as e:
                    print(f"Error classifying segment '{segment}': {e}")
                    entities.append([start, end, segment, "unassigned"])
            
            # Handle repeated words (mark the second occurrence as unassigned)
            entities = detect_repeated_words(address, entities)
            
            # Sort entities by position
            entities.sort(key=lambda x: x[0])
            
            # Store the entities
            address_data["entities"] = entities
        
        # Save progress after each batch
        print(f"Completed batch {batch_idx//batch_size + 1}/{len(addresses)//batch_size + 1}")
        with open(f'zero_shot_annotated_batch_{batch_idx}.json', 'w', encoding='utf-8') as f:
            json.dump(batch, f, indent=2, ensure_ascii=False)
    
    # Combine all batches
    all_annotated = []
    for batch_idx in range(0, len(addresses), batch_size):
        with open(f'zero_shot_annotated_batch_{batch_idx}.json', 'r', encoding='utf-8') as f:
            batch = json.load(f)
            all_annotated.extend(batch)
    
    # Save all annotated data
    with open('zero_shot_annotated_ner_train.json', 'w', encoding='utf-8') as f:
        json.dump(all_annotated, f, indent=2, ensure_ascii=False)
    
    return all_annotated

if __name__ == "__main__":
    # Load addresses from CSV or JSON
    try:
        # Try loading JSON if it exists
        with open('new_ner_train.json', 'r', encoding='utf-8') as f:
            addresses = json.load(f)
        print(f"Loaded {len(addresses)} addresses from JSON")
    except FileNotFoundError:
        # Otherwise load from CSV
        df = pd.read_csv('Assurekit_final.csv')
        addresses = []
        for _, row in df.iterrows():
            if not pd.isna(row['customerAddress']):
                address = clean_address(row['customerAddress'])
                if address:  # Skip empty addresses
                    addresses.append({"text": address, "entities": []})
        print(f"Loaded {len(addresses)} addresses from CSV")
        # Save the cleaned addresses
        with open('new_ner_train.json', 'w', encoding='utf-8') as f:
            json.dump(addresses, f, indent=2, ensure_ascii=False)
    
    # Annotate addresses
    batch_size = 10  # Process 10 addresses at a time
    annotated_addresses = zero_shot_ner(addresses, batch_size)
    
    # Print statistics
    entity_counts = {}
    addresses_with_entities = 0
    
    for address in annotated_addresses:
        if address["entities"]:
            addresses_with_entities += 1
            
        for entity in address["entities"]:
            entity_type = entity[3]
            entity_counts[entity_type] = entity_counts.get(entity_type, 0) + 1
    
    print("\nAnnotation Statistics:")
    print(f"Total addresses: {len(annotated_addresses)}")
    print(f"Addresses with entities: {addresses_with_entities} ({addresses_with_entities/len(annotated_addresses)*100:.1f}%)")
    
    print("\nEntity Type Distribution:")
    for entity_type, count in sorted(entity_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"{entity_type}: {count}")
    
    print("\nAnnotation complete! Results saved to zero_shot_annotated_ner_train.json")

Loaded 99 addresses from JSON
Loading zero-shot classification model...


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3d2652e7-6296-4354-9f2c-49ba40d08522)')' thrown while requesting HEAD https://huggingface.co/facebook/bart-large-mnli/resolve/main/config.json
Retrying in 1s [Retry 1/5].
Device set to use cpu


Processing 99 addresses in batches of 10


 10%|█         | 1/10 [00:38<05:45, 38.41s/it]

Completed batch 1/10


 20%|██        | 2/10 [01:10<04:36, 34.60s/it]

Completed batch 2/10


 20%|██        | 2/10 [01:12<04:49, 36.21s/it]


KeyboardInterrupt: 

In [14]:
pip install datasets

Defaulting to user installation because normal site-packages is not writeable
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
Using cached dill-0.4.0-py3-none-any.whl (119 kB)
Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.2/34.2 MB 12.2 MB/s  0:00:02 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [datasets]/12 [datasets]ess]

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Bert Base Cases

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer
import json

with open("ner_list_train.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

data = []
for item in raw_data:
    entities = []
    for ent in item["entities"]:
        start, end, text, label = ent
        entities.append({
            "start": int(start),
            "end": int(end),
            "text": text,
            "label": label
        })
    data.append({
        "text": item["text"],
        "entities": entities
    })

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

label_list = [
    "O",
    "B-unit", "I-unit",
    "B-street_name", "I-street_name",
    "B-society_name", "I-society_name",
    "B-area_name", "I-area_name",
    "B-city_name", "I-city_name",
    "B-state_name", "I-state_name",
    "B-landmark", "I-landmark"
]

label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}


def encode_examples(example):
    text = example["text"]
    entities = example["entities"]

    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    labels = ["O"] * len(tokenized["input_ids"])

    for ent in entities:
        start, end, label_name = ent["start"], ent["end"], ent["label"]
        for i, (offset_start, offset_end) in enumerate(tokenized["offset_mapping"]):
            # skip special tokens
            if offset_start == offset_end:
                continue
            if offset_start >= start and offset_end <= end:
                prefix = "B-" if labels[i] == "O" else "I-"
                if f"{prefix}{label_name}" in label2id:
                    labels[i] = f"{prefix}{label_name}"

    tokenized["labels"] = [label2id[label] for label in labels]
    tokenized.pop("offset_mapping")
    return tokenized


dataset = Dataset.from_list(data)

tokenized_dataset = dataset.map(encode_examples)

tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

print(tokenized_dataset)


Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'entities', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 10800
    })
    test: Dataset({
        features: ['text', 'entities', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 1200
    })
})


In [9]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16250 sha256=38a5af08d0149d302dc2372f31bc7899b121564331aeda2b14e290c15df9c6fe
  Stored in directory: /Users/sarang/Library/Caches/pip/wheels/14/cf/a7/8f28ef376d707ff10e3922899482a2f23ef3002f8a952f47ac
Successfully built seqeval

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install -U transformers datasets evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install 'accelerate>=0.26.0'

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-cased",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id
)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

training_args = TrainingArguments(
    output_dir="./address_ner_model",
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.save_model("./address_ner_model")
tokenizer.save_pretrained("./address_ner_model")


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_19910/222278755.py:49: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


('./address_ner_model/tokenizer_config.json',
 './address_ner_model/special_tokens_map.json',
 './address_ner_model/vocab.txt',
 './address_ner_model/added_tokens.json',
 './address_ner_model/tokenizer.json')

Tokenized Output

In [ ]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner",
    model="./address_ner_model",
    tokenizer="./address_ner_model",
    aggregation_strategy="simple"
)

sample = "flat 24A, palm springs near indiranagar metro, bengaluru"
preds = ner_pipeline(sample)

for p in preds:
    entity = p['word'].replace("##", "")
    print(f"{entity:40s} → {p['entity_group']:15s} ({p['score']:.2f})")

Device set to use mps:0


flat                                     → unit            (1.00)
24                                       → unit            (1.00)
A                                        → unit            (1.00)
palm                                     → unit            (0.59)
springs                                  → unit            (0.33)
near                                     → landmark        (1.00)
in                                       → landmark        (1.00)
dira                                     → landmark        (1.00)
nagar                                    → landmark        (1.00)
metro                                    → landmark        (1.00)
ben                                      → city_name       (0.95)
gal                                      → city_name       (0.96)
uru                                      → city_name       (0.94)


In [6]:
import pandas as pd

df = pd.DataFrame({
    "address": [
        "flat 24A, palm springs near indiranagar metro, bengaluru",
        "no 5, brigade road, mg road, bangalore",
        "12A lakshmi towers, velachery, chennai"
    ]
})

df["ner_predictions"] = df["address"].apply(lambda x: ner_pipeline(x))

Entity level Output

In [8]:
def extract_entities(text):
    preds = ner_pipeline(text)
    entities = {}
    current_entity = None
    buffer = []

    for p in preds:
        ent = p["entity_group"]
        word = p["word"]
        # Remove subword marker
        if word.startswith("##"):
            word = word[2:]
            buffer[-1] += word  # append to previous subword
        else:
            if current_entity and ent != current_entity:
                # Commit buffer to entities dict
                entities.setdefault(current_entity, []).append(" ".join(buffer))
                buffer = []
            buffer.append(word)
            current_entity = ent

    # Commit last buffered entity
    if buffer:
        entities.setdefault(current_entity, []).append(" ".join(buffer))

    return {k: " ".join(v) for k, v in entities.items()}

print(extract_entities("flat 24A, palm springs near indiranagar metro, bengaluru"))

{'unit': 'flat 24A palm springs', 'landmark': 'near indiranagar metro', 'city_name': 'bengaluru'}


Eval snippet

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer
import evaluate
import numpy as np
import json
from datasets import Dataset

model_path = "./address_ner_model"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

label_list = [
    "O",
    "B-unit", "I-unit",
    "B-street_name", "I-street_name",
    "B-society_name", "I-society_name",
    "B-area_name", "I-area_name",
    "B-city_name", "I-city_name",
    "B-state_name", "I-state_name",
    "B-landmark", "I-landmark"
]

label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

with open("ner_list_test.json", "r", encoding="utf-8") as f:
    raw_test = json.load(f)

data = []
for item in raw_test:
    entities = []
    for ent in item["entities"]:
        start, end, text, label = ent
        entities.append({
            "start": int(start),
            "end": int(end),
            "text": text,
            "label": label
        })
    data.append({
        "text": item["text"],
        "entities": entities
    })

test_dataset = Dataset.from_list(data)

def encode_examples(example):
    text = example["text"]
    entities = example["entities"]

    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    labels = ["O"] * len(tokenized["input_ids"])

    for ent in entities:
        start, end, label_name = ent["start"], ent["end"], ent["label"]
        for i, (offset_start, offset_end) in enumerate(tokenized["offset_mapping"]):
            if offset_start == offset_end:
                continue
            if offset_start >= start and offset_end <= end:
                prefix = "B-" if labels[i] == "O" else "I-"
                if f"{prefix}{label_name}" in label2id:
                    labels[i] = f"{prefix}{label_name}"

    tokenized["labels"] = [label2id[label] for label in labels]
    tokenized.pop("offset_mapping")
    return tokenized

tokenized_test = test_dataset.map(encode_examples)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

results = trainer.evaluate(eval_dataset=tokenized_test)
print("\n📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k:<15s}: {v:.4f}")

Map:   0%|          | 0/2199 [00:00<?, ? examples/s]

/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_19910/4155517150.py:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



📊 Evaluation Results:
eval_loss      : 0.1075
eval_model_preparation_time: 0.0013
eval_precision : 0.7661
eval_recall    : 0.7603
eval_f1        : 0.7632
eval_accuracy  : 0.9692
eval_runtime   : 34.8135
eval_samples_per_second: 63.1650
eval_steps_per_second: 7.8990


BERT BASE NER

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import json

with open("ner_list_train.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

data = []
for item in raw_data:
    entities = []
    for ent in item["entities"]:
        start, end, text, label = ent
        entities.append({
            "start": int(start),
            "end": int(end),
            "text": text,
            "label": label
        })
    data.append({
        "text": item["text"],
        "entities": entities
    })

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

label_list = [
    "O",
    "B-unit", "I-unit",
    "B-street_name", "I-street_name",
    "B-society_name", "I-society_name",
    "B-area_name", "I-area_name",
    "B-city_name", "I-city_name",
    "B-state_name", "I-state_name",
    "B-landmark", "I-landmark"
]

label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

def encode_examples(example):
    text = example["text"]
    entities = example["entities"]

    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    labels = ["O"] * len(tokenized["input_ids"])

    for ent in entities:
        start, end, label_name = ent["start"], ent["end"], ent["label"]
        for i, (offset_start, offset_end) in enumerate(tokenized["offset_mapping"]):
            if offset_start == offset_end:
                continue
            if offset_start >= start and offset_end <= end:
                prefix = "B-" if labels[i] == "O" else "I-"
                if f"{prefix}{label_name}" in label2id:
                    labels[i] = f"{prefix}{label_name}"

    tokenized["labels"] = [label2id[label] for label in labels]
    tokenized.pop("offset_mapping")
    return tokenized

dataset = Dataset.from_list(data)
tokenized_dataset = dataset.map(encode_examples)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

model = AutoModelForTokenClassification.from_pretrained(
    "dslim/bert-base-NER",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

# --- Step 8: Training arguments ---
training_args = TrainingArguments(
    output_dir="./address_ner_model_bertNER",
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
)

# --- Step 9: Trainer ---
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# --- Step 10: Train and Save ---
trainer.train()
trainer.save_model("./address_ner_model_bertNER")
tokenizer.save_pretrained("./address_ner_model_bertNER")

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dslim/bert-base-NER and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([9]) in the checkpoint and torch.Size([15]) in the model instantiated
- classifier.weight: found shape torch.Size([9, 768])

/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


('./address_ner_model_bertNER/tokenizer_config.json',
 './address_ner_model_bertNER/special_tokens_map.json',
 './address_ner_model_bertNER/vocab.txt',
 './address_ner_model_bertNER/added_tokens.json',
 './address_ner_model_bertNER/tokenizer.json')

In [29]:
test_dataset

Dataset({
    features: ['text', 'entities'],
    num_rows: 2199
})

Tokenized Output

In [ ]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner",
    model="./address_ner_model_bertNER",
    tokenizer="./address_ner_model_bertNER",
    aggregation_strategy="simple"
)

sample = "flat 24A, palm springs near indiranagar metro, bengaluru, karnataka, 827004"
preds = ner_pipeline(sample)

for p in preds:
    entity = p['word'].replace("##", "")
    print(f"{entity:40s} → {p['entity_group']:15s} ({p['score']:.2f})")

Device set to use mps:0


flat                                     → unit            (1.00)
24                                       → unit            (1.00)
A                                        → unit            (1.00)
palm                                     → landmark        (0.92)
springs                                  → landmark        (0.99)
near                                     → landmark        (1.00)
in                                       → landmark        (0.99)
dira                                     → landmark        (0.99)
nagar                                    → landmark        (0.98)
metro                                    → landmark        (0.99)
ben                                      → city_name       (0.72)
gal                                      → city_name       (0.85)
uru                                      → city_name       (0.90)
ka                                       → state_name      (0.97)
rna                                      → state_name      (0.96)
ta        

Entity level Output

In [ ]:
def extract_entities(text):
    preds = ner_pipeline(text)
    entities = {}
    current_entity = None
    buffer = []

    for p in preds:
        ent = p["entity_group"]
        word = p["word"]
        # Remove subword marker
        if word.startswith("##"):
            word = word[2:]
            buffer[-1] += word  # append to previous subword
        else:
            if current_entity and ent != current_entity:
                # Commit buffer to entities dict
                entities.setdefault(current_entity, []).append(" ".join(buffer))
                buffer = []
            buffer.append(word)
            current_entity = ent

    # Commit last buffered entity
    if buffer:
        entities.setdefault(current_entity, []).append(" ".join(buffer))

    return {k: " ".join(v) for k, v in entities.items()}

print(extract_entities("flat 24A, palm springs near indiranagar metro, bengaluru, karnataka"))

{'unit': 'flat 24A', 'landmark': 'palm springs near indiranagar metro', 'city_name': 'bengaluru', 'state_name': 'karnataka'}


In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer
import evaluate
import numpy as np
import json
from datasets import Dataset

model_path = "./address_ner_model_bertNER"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

label_list = [
    "O",
    "B-unit", "I-unit",
    "B-street_name", "I-street_name",
    "B-society_name", "I-society_name",
    "B-area_name", "I-area_name",
    "B-city_name", "I-city_name",
    "B-state_name", "I-state_name",
    "B-landmark", "I-landmark"
]

label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

with open("ner_list_test.json", "r", encoding="utf-8") as f:
    raw_test = json.load(f)

data = []
for item in raw_test:
    entities = []
    for ent in item["entities"]:
        start, end, text, label = ent
        entities.append({
            "start": int(start),
            "end": int(end),
            "text": text,
            "label": label
        })
    data.append({
        "text": item["text"],
        "entities": entities
    })

test_dataset = Dataset.from_list(data)

def encode_examples(example):
    text = example["text"]
    entities = example["entities"]

    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    labels = ["O"] * len(tokenized["input_ids"])

    for ent in entities:
        start, end, label_name = ent["start"], ent["end"], ent["label"]
        for i, (offset_start, offset_end) in enumerate(tokenized["offset_mapping"]):
            if offset_start == offset_end:
                continue
            if offset_start >= start and offset_end <= end:
                prefix = "B-" if labels[i] == "O" else "I-"
                if f"{prefix}{label_name}" in label2id:
                    labels[i] = f"{prefix}{label_name}"

    tokenized["labels"] = [label2id[label] for label in labels]
    tokenized.pop("offset_mapping")
    return tokenized

tokenized_test = test_dataset.map(encode_examples)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

results = trainer.evaluate(eval_dataset=tokenized_test)
print("\n📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k:<15s}: {v:.4f}")


Map:   0%|          | 0/2199 [00:00<?, ? examples/s]

/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_19910/2441458831.py:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)



📊 Evaluation Results:
eval_loss      : 0.1092
eval_model_preparation_time: 0.0015
eval_precision : 0.7618
eval_recall    : 0.7583
eval_f1        : 0.7601
eval_accuracy  : 0.9688
eval_runtime   : 44.3548
eval_samples_per_second: 49.5770
eval_steps_per_second: 6.2000


TinyBERT

In [3]:
pip install -U transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import json

with open("ner_list_train.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

data = []
for item in raw_data:
    entities = []
    for ent in item["entities"]:
        start, end, text, label = ent
        entities.append({
            "start": int(start),
            "end": int(end),
            "text": text,
            "label": label
        })
    data.append({
        "text": item["text"],
        "entities": entities
    })

# --- Step 3: Tokenizer and label mapping ---
tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_6L_768D")

label_list = [
    "O",
    "B-unit", "I-unit",
    "B-street_name", "I-street_name",
    "B-society_name", "I-society_name",
    "B-area_name", "I-area_name",
    "B-city_name", "I-city_name",
    "B-state_name", "I-state_name",
    "B-landmark", "I-landmark"
]

label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

def encode_examples(example):
    text = example["text"]
    entities = example["entities"]

    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    labels = ["O"] * len(tokenized["input_ids"])

    for ent in entities:
        start, end, label_name = ent["start"], ent["end"], ent["label"]
        for i, (offset_start, offset_end) in enumerate(tokenized["offset_mapping"]):
            if offset_start == offset_end:
                continue
            if offset_start >= start and offset_end <= end:
                prefix = "B-" if labels[i] == "O" else "I-"
                if f"{prefix}{label_name}" in label2id:
                    labels[i] = f"{prefix}{label_name}"

    tokenized["labels"] = [label2id[label] for label in labels]
    tokenized.pop("offset_mapping")
    return tokenized

dataset = Dataset.from_list(data)
tokenized_dataset = dataset.map(encode_examples)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

model = AutoModelForTokenClassification.from_pretrained(
    "huawei-noah/TinyBERT_General_6L_768D",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

training_args = TrainingArguments(
    output_dir="./address_ner_model_TinyBERT",
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("./address_ner_model_TinyBERT")
tokenizer.save_pretrained("./address_ner_model_TinyBERT")

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at huawei-noah/TinyBERT_General_6L_768D and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_1530/3914674292.py:124: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


('./address_ner_model_TinyBERT/tokenizer_config.json',
 './address_ner_model_TinyBERT/special_tokens_map.json',
 './address_ner_model_TinyBERT/vocab.txt',
 './address_ner_model_TinyBERT/added_tokens.json',
 './address_ner_model_TinyBERT/tokenizer.json')

Tokenized Output

In [2]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner",
    model="./address_ner_model_TinyBERT",
    tokenizer="./address_ner_model_TinyBERT",
    aggregation_strategy="simple"
)

sample = "flat 24A, palm springs near indiranagar metro, bengaluru, karnataka, 827004"
preds = ner_pipeline(sample)

# --- Post-process and display ---
for p in preds:
    entity = p['word'].replace("##", "")
    print(f"{entity:40s} → {p['entity_group']:15s} ({p['score']:.2f})")

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


flat                                     → unit            (0.99)
24                                       → unit            (0.99)
a                                        → unit            (0.99)
palm                                     → unit            (0.82)
springs                                  → unit            (0.74)
near                                     → landmark        (0.99)
indira                                   → landmark        (0.99)
nagar                                    → landmark        (0.99)
metro                                    → landmark        (0.99)
bengal                                   → city_name       (0.68)
uru                                      → city_name       (0.70)
karnataka                                → state_name      (0.94)


Entity level Outputs

In [3]:
def extract_entities(text):
    preds = ner_pipeline(text)
    entities = {}
    current_entity = None
    buffer = []

    for p in preds:
        ent = p["entity_group"]
        word = p["word"]
        # Remove subword marker
        if word.startswith("##"):
            word = word[2:]
            buffer[-1] += word  # append to previous subword
        else:
            if current_entity and ent != current_entity:
                # Commit buffer to entities dict
                entities.setdefault(current_entity, []).append(" ".join(buffer))
                buffer = []
            buffer.append(word)
            current_entity = ent

    # Commit last buffered entity
    if buffer:
        entities.setdefault(current_entity, []).append(" ".join(buffer))

    return {k: " ".join(v) for k, v in entities.items()}

print(extract_entities("flat 24A, palm springs near indiranagar metro, bengaluru, karnataka"))

{'unit': 'flat 24a palm springs', 'landmark': 'near indiranagar metro', 'city_name': 'bengaluru', 'state_name': 'karnataka'}


Eval code snippet

In [ ]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer
import evaluate
import numpy as np
import json
from datasets import Dataset

model_path = "./address_ner_model_TinyBERT"
model = AutoModelForTokenClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

label_list = [
    "O",
    "B-unit", "I-unit",
    "B-street_name", "I-street_name",
    "B-society_name", "I-society_name",
    "B-area_name", "I-area_name",
    "B-city_name", "I-city_name",
    "B-state_name", "I-state_name",
    "B-landmark", "I-landmark"
]

label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

with open("ner_list_test.json", "r", encoding="utf-8") as f:
    raw_test = json.load(f)

data = []
for item in raw_test:
    entities = []
    for ent in item["entities"]:
        start, end, text, label = ent
        entities.append({
            "start": int(start),
            "end": int(end),
            "text": text,
            "label": label
        })
    data.append({
        "text": item["text"],
        "entities": entities
    })

test_dataset = Dataset.from_list(data)

def encode_examples(example):
    text = example["text"]
    entities = example["entities"]

    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    labels = ["O"] * len(tokenized["input_ids"])

    for ent in entities:
        start, end, label_name = ent["start"], ent["end"], ent["label"]
        for i, (offset_start, offset_end) in enumerate(tokenized["offset_mapping"]):
            if offset_start == offset_end:
                continue
            if offset_start >= start and offset_end <= end:
                prefix = "B-" if labels[i] == "O" else "I-"
                if f"{prefix}{label_name}" in label2id:
                    labels[i] = f"{prefix}{label_name}"

    tokenized["labels"] = [label2id[label] for label in labels]
    tokenized.pop("offset_mapping")
    return tokenized

tokenized_test = test_dataset.map(encode_examples)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# --- Step 7: Evaluate ---
results = trainer.evaluate(eval_dataset=tokenized_test)
print("\n📊 Evaluation Results:")
for k, v in results.items():
    print(f"{k:<15s}: {v:.4f}")


Map:   0%|          | 0/2199 [00:00<?, ? examples/s]

/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_1530/2610826993.py:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/sarang/Library/Python/3.13/lib/python/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)



📊 Evaluation Results:
eval_loss      : 0.0854
eval_model_preparation_time: 0.0006
eval_precision : 0.7609
eval_recall    : 0.7561
eval_f1        : 0.7585
eval_accuracy  : 0.9727
eval_runtime   : 19.9022
eval_samples_per_second: 110.4900
eval_steps_per_second: 13.8180


Training TinyBERT on top of our whole corpus

In [6]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer
import evaluate
import numpy as np
import json

with open("ner_address_corpus.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

data = []
for item in raw_data:
    entities = []
    for ent in item["entities"]:
        start, end, text, label = ent
        entities.append({
            "start": int(start),
            "end": int(end),
            "text": text,
            "label": label
        })
    data.append({
        "text": item["text"],
        "entities": entities
    })

tokenizer = AutoTokenizer.from_pretrained("huawei-noah/TinyBERT_General_6L_768D")

label_list = [
    "O",
    "B-unit", "I-unit",
    "B-street_name", "I-street_name",
    "B-society_name", "I-society_name",
    "B-area_name", "I-area_name",
    "B-city_name", "I-city_name",
    "B-state_name", "I-state_name",
    "B-landmark", "I-landmark",
    "B-area_pincode", "I-area_pincode",
    "B-unassigned", "I-unassigned"
]

label2id = {label: idx for idx, label in enumerate(label_list)}
id2label = {idx: label for label, idx in label2id.items()}

def encode_examples(example):
    text = example["text"]
    entities = example["entities"]

    tokenized = tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_offsets_mapping=True
    )

    labels = ["O"] * len(tokenized["input_ids"])

    for ent in entities:
        start, end, label_name = ent["start"], ent["end"], ent["label"]
        for i, (offset_start, offset_end) in enumerate(tokenized["offset_mapping"]):
            if offset_start == offset_end:
                continue
            if offset_start >= start and offset_end <= end:
                prefix = "B-" if labels[i] == "O" else "I-"
                if f"{prefix}{label_name}" in label2id:
                    labels[i] = f"{prefix}{label_name}"

    tokenized["labels"] = [label2id[label] for label in labels]
    tokenized.pop("offset_mapping")
    return tokenized

dataset = Dataset.from_list(data)
tokenized_dataset = dataset.map(encode_examples)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.1)

model = AutoModelForTokenClassification.from_pretrained(
    "huawei-noah/TinyBERT_General_6L_768D",
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)

metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

training_args = TrainingArguments(
    output_dir="./address_ner_model_TinyBERT2",
    do_train=True,
    do_eval=True,
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500,
    eval_steps=500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
trainer.save_model("./address_ner_model_TinyBERT2")
tokenizer.save_pretrained("./address_ner_model_TinyBERT2")

FileNotFoundError: [Errno 2] No such file or directory: 'ner_address_corpus.json'

Tokenized Output

In [32]:
from transformers import pipeline

ner_pipeline = pipeline(
    "ner",
    model="./address_ner_model_TinyBERT2",
    tokenizer="./address_ner_model_TinyBERT2",
    aggregation_strategy="simple"
)

sample = "C-5 BLDG-11 FLAT NO 3 4TH FLOOR SECTOR-1A CBD BELAPUR NAVI MUMBAI MANGO GARDEN ENCLAVE THANE 400614 MH,MH - 400614"
preds = ner_pipeline(sample)

# --- Post-process and display ---
for p in preds:
    entity = p['word'].replace("##", "")
    print(f"{entity:40s} → {p['entity_group']:15s} ({p['score']:.2f})")

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


c                                        → unit            (0.99)
-                                        → unit            (0.96)
5                                        → unit            (0.99)
b                                        → unit            (0.96)
ld                                       → unit            (0.94)
g                                        → unit            (0.95)
-                                        → unit            (0.90)
11                                       → unit            (0.98)
flat                                     → unit            (0.99)
no                                       → unit            (0.99)
3                                        → unit            (0.98)
4th                                      → unit            (0.98)
floor                                    → unit            (0.98)
sector                                   → unit            (0.92)
-                                        → unit            (0.68)
1a        

Entity level Outputs

In [37]:
def extract_entities(text):
    preds = ner_pipeline(text)
    entities = {}
    current_entity = None
    buffer = []

    for p in preds:
        ent = p["entity_group"]
        word = p["word"]
        # Remove subword marker
        if word.startswith("##"):
            word = word[2:]
            buffer[-1] += word  # append to previous subword
        else:
            if current_entity and ent != current_entity:
                # Commit buffer to entities dict
                entities.setdefault(current_entity, []).append(" ".join(buffer))
                buffer = []
            buffer.append(word)
            current_entity = ent

    # Commit last buffered entity
    if buffer:
        entities.setdefault(current_entity, []).append(" ".join(buffer))

    return {k: " ".join(v) for k, v in entities.items()}

print(extract_entities("C-5 BLDG-11 FLAT NO 3 4TH FLOOR SECTOR-1A CBD BELAPUR NAVI MUMBAI MANGO GARDEN ENCLAVE THANE 400614 MH,MH - 400614"))

{'unit': 'c - 5 bldg - 11 flat no 3 4th floor sector - 1a', 'area_name': 'cbd belapur', 'city_name': 'navi mumbai thane', 'society_name': 'mango garden enclave', 'area_pincode': '400614 mh', 'unassigned': 'mh 400614'}


List of strings for each entity

In [ ]:
def extract_entities(text):
    preds = ner_pipeline(text)
    entities = {}
    current = None

    for p in preds:
        ent = p["entity_group"]
        start = p["start"]
        end = p["end"]

        if not current or current["entity"] != ent or start > current["end"] + 2:
            if current:
                span_text = text[current["start"]:current["end"]].strip()
                entities.setdefault(current["entity"], []).append(span_text)
            current = {"entity": ent, "start": start, "end": end}
        else:
            current["end"] = end  # extend current span

    # Commit last entity
    if current:
        span_text = text[current["start"]:current["end"]].strip()
        entities.setdefault(current["entity"], []).append(span_text)

    return entities


sample = "C-5 BLDG-11 FLAT NO 3 4TH FLOOR SECTOR-1A CBD BELAPUR NAVI MUMBAI MANGO GARDEN ENCLAVE THANE 400614 MH,MH - 400614"
print(extract_entities(sample))

{'unit': ['C-5 BLDG-11 FLAT NO 3 4TH FLOOR SECTOR-1A'], 'area_name': ['CBD BELAPUR'], 'city_name': ['NAVI MUMBAI', 'THANE', 'H'], 'society_name': ['MANGO GARDEN ENCLAVE'], 'area_pincode': ['400614 M'], 'unassigned': ['MH', '400614']}


In [ ]:
# import pandas as pd
# import json
# from transformers import pipeline

# # --- 1️⃣ Load model + tokenizer ---
# ner_pipeline = pipeline(
#     "ner",
#     model="./address_ner_model_TinyBERT1",
#     tokenizer="./address_ner_model_TinyBERT1",
#     aggregation_strategy=None  # disable auto-aggregation to manually merge properly
# )

# # --- 2️⃣ Load CSV ---
# df = pd.read_csv("Assurekit_final_completeness.csv")
# df = df.head(10)

# # --- 3️⃣ Subword merge helper ---
# def merge_subwords(preds, text):
#     merged_entities = []
#     if not preds:
#         return merged_entities

#     current = preds[0].copy()
#     for p in preds[1:]:
#         # Same entity type & contiguous text -> merge
#         if p["entity"] == current["entity"] and p["start"] <= current["end"]:
#             current["end"] = p["end"]
#             current["word"] += text[p["start"]:p["end"]]
#         else:
#             merged_entities.append(current)
#             current = p.copy()
#     merged_entities.append(current)
#     return merged_entities

# # --- 4️⃣ Get entity spans per address ---
# def get_entity_spans(text):
#     preds = ner_pipeline(text)
#     preds = merge_subwords(preds, text)

#     entities = []
#     for p in preds:
#         start, end = p["start"], p["end"]
#         entity_text = text[start:end].strip()
#         label = p["entity"].split("-")[-1] if "-" in p["entity"] else p["entity"]
#         entities.append([start, end, entity_text, label])
#     return entities

# # --- 5️⃣ Annotate all rows ---
# annotated_data = []

# for idx, row in df.iterrows():
#     text = str(row["customerAddress"]).strip()
#     if not text or text.lower() == "nan":
#         continue
#     try:
#         entities = get_entity_spans(text)
#         annotated_data.append({
#             "text": text,
#             "entities": entities
#         })
#     except Exception as e:
#         print(f"⚠️ Error processing row {idx}: {e}")

# # --- 6️⃣ Save JSON output ---
# output_file = "Assurekit_final_annotated_clean.json"
# with open(output_file, "w", encoding="utf-8") as f:
#     json.dump(annotated_data, f, ensure_ascii=False, indent=4)

# print(f"✅ Clean entity-level annotations saved to {output_file}")
# print(f"🧾 Total addresses processed: {len(annotated_data)}")

Device set to use mps:0
/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_1127/655489210.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Assurekit_final_completeness.csv")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Clean entity-level annotations saved to Assurekit_final_annotated_clean.json
🧾 Total addresses processed: 10


In [ ]:
# import pandas as pd
# import json
# from transformers import pipeline

# # --- 1️⃣ Load your trained model ---
# ner_pipeline = pipeline(
#     "ner",
#     model="./address_ner_model_TinyBERT1",
#     tokenizer="./address_ner_model_TinyBERT1",
#     aggregation_strategy="simple"
# )

# # --- 2️⃣ Load the CSV file ---
# df = pd.read_csv("Assurekit_final_completeness.csv")
# df = df.head(10)

# # Sanity check
# if "customerAddress" not in df.columns:
#     raise ValueError("Column 'customerAddress' not found in CSV!")

# # --- 3️⃣ Define a function to extract entity spans ---
# def get_entity_spans(text):
#     preds = ner_pipeline(text)
#     entities = []
#     for p in preds:
#         start = p["start"]
#         end = p["end"]
#         word = text[start:end]
#         label = p["entity_group"]
#         entities.append([start, end, word, label])
#     return entities

# # --- 4️⃣ Build the annotated data list ---
# annotated_data = []

# for idx, row in df.iterrows():
#     text = str(row["customerAddress"]).strip()
#     if not text or text.lower() == "nan":
#         continue  # skip empty rows
#     try:
#         entities = get_entity_spans(text)
#         annotated_data.append({
#             "text": text,
#             "entities": entities
#         })
#     except Exception as e:
#         print(f"⚠️ Error processing row {idx}: {e}")

# # --- 5️⃣ Save the annotated data as JSON ---
# output_file = "Assurekit_final_annotated.json"
# with open(output_file, "w", encoding="utf-8") as f:
#     json.dump(annotated_data, f, ensure_ascii=False, indent=4)

# print(f"✅ Annotation completed! Saved to {output_file}")
# print(f"🧾 Total annotated addresses: {len(annotated_data)}")

Device set to use mps:0
/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_1127/3082235961.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Assurekit_final_completeness.csv")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Annotation completed! Saved to Assurekit_final_annotated.json
🧾 Total annotated addresses: 10


In [ ]:
import pandas as pd
import json
import re
import ast
from transformers import pipeline


def clean_address(address):
    """Clean and preprocess address text"""
    address = re.sub(r"(https?://\S+)", "", address)
    address = re.sub(r"[\n\t\r]", " ", address)
    address = re.sub(r"[\'\"\`]s", "s", address)
    address = re.sub(r"[\'\"] ", " ", address)
    address = re.sub(r" [\'\"]", " ", address)
    address = re.sub(r"[a-zA-Z]{2,}[\'\"][a-zA-Z]{2,}", " ", address)
    address = re.sub(r"[\=\:\~]", "-", address)
    address = re.sub(r"[\[\{]", "(", address)
    address = re.sub(r"[\]\}]", ")", address)
    address = re.sub(r"[\|\\]", "/", address)
    address = re.sub(r"[;\?]", ",", address)
    address = re.sub(r"[`\!\$@\*%\<\>_\^]", " ", address)
    # Rule11: collapse repeated symbols
    for ch in [",", ".", "+", "-", "(", ")", "&", "#", "/", "\"", "'"]:
        address = re.sub(f"\\{ch}+", ch, address)
    address = re.sub(" +", " ", address)
    # Rule12: trim edges
    address = address.strip()
    address = re.sub(r"^[\.\,\-\+\/\)]", "", address)
    address = re.sub(r"[\.\,\-\+\/\(]$", "", address)
    address = address.strip()
    # Rule13: separate punctuation from end of tokens
    parts = []
    for token in address.split():
        match_ = re.search("[\\\\.,;:\\-_]+$", token)
        if match_:
            token = re.sub("[\\\\.,;:\\-_]+$", " " + match_.group(0), token)
        parts.append(token)
    address = ' '.join(parts)
    # Lowercase final text
    address = address.lower()
    return address

ner_pipeline = pipeline(
    "ner",
    model="./address_ner_model_TinyBERT2",
    tokenizer="./address_ner_model_TinyBERT2",
    aggregation_strategy=None  # subword merging handled manually
)

def merge_subwords(preds):
    if not preds:
        return []

    merged = []
    current = preds[0].copy()

    for p in preds[1:]:
        if (
            p["entity"] == current["entity"]
            and p["start"] <= current["end"] + 1
        ):
            current["end"] = p["end"]
            current["word"] += " " + p["word"].replace("##", "")
        else:
            merged.append(current)
            current = p.copy()
    merged.append(current)
    return merged

def get_entity_spans(text):
    preds = ner_pipeline(text)
    preds = merge_subwords(preds)

    entities = []
    for p in preds:
        start, end = p["start"], p["end"]
        label = p["entity"].split("-")[-1] if "-" in p["entity"] else p["entity"]
        entity_text = text[start:end].strip(" ,.")
        entities.append([start, end, entity_text, label])
    return entities

df = pd.read_csv("Assurekit_final_completeness.csv")
df = df.head(20)


annotated_data = []
for idx, row in df.iterrows():
    raw_text = str(row.get("customerAddress", "")).strip()
    if not raw_text or raw_text.lower() == "nan":
        continue

    # Clean before annotation
    text = clean_address(raw_text)

    try:
        entities = get_entity_spans(text)
        annotated_data.append({
            "text": text,
            "entities": entities
        })
    except Exception as e:
        print(f"⚠️ Error processing row {idx}: {e}")


output_file = "Assurekit_final_annotated.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(annotated_data, f, ensure_ascii=False, indent=4)

print(f"✅ Entity-level annotations (cleaned) saved to {output_file}")
print(f"🧾 Total addresses processed: {len(annotated_data)}")


Device set to use mps:0
/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_1127/417052366.py:96: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Assurekit_final_completeness.csv")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Entity-level annotations (cleaned) saved to Assurekit_final_annotated.json
🧾 Total addresses processed: 20


Final bulk level logic

In [10]:
import pandas as pd
import json
import re
import ast
from transformers import pipeline
from fuzzywuzzy import process


VALID_CITIES = [
    # Tier 1 Cities
    "mumbai", "pune", "bengaluru", "bangalore", "chennai", "hyderabad", "delhi",
    "gurugram", "gurgaon", "noida", "kolkata", "ahmedabad", "surat", "jaipur",
    "lucknow", "indore", "bhopal", "nagpur", "chandigarh", "coimbatore", "visakhapatnam",
    "vijayawada", "patna", "ranchi", "kochi", "trivandrum", "madurai", "mysore",

    # Tier 2 / 3 Cities
    "amritsar", "jalandhar", "ludhiana", "varanasi", "kanpur", "allahabad",
    "meerut", "dehradun", "shimla", "haridwar", "patiala", "nashik", "aurangabad",
    "kolhapur", "rajkot", "vadodara", "gandhinagar", "bhavnagar", "jamnagar",
    "udaipur", "jodhpur", "ajmer", "bikaner", "alwar", "gwalior", "sagar",
    "rewa", "satna", "bilaspur", "raipur", "durg", "bhilai", "cuttack",
    "bhubaneswar", "puri", "rourkela", "silchar", "dibrugarh", "guwahati",
    "imphal", "aizawl", "shillong", "agartala", "itanagar", "gangtok",
    "panaji", "margao", "mapusa", "porvorim", "pondicherry", "karaikal",
    "tiruchirappalli", "salem", "vellore", "tirunelveli", "nagercoil",
    "eranakulam", "thrissur", "palakkad", "malappuram", "alappuzha",
    "hosur", "tumkur", "hubli", "dharwad", "belgaum", "mangalore",
    "bellary", "shimoga", "davangere", "chitradurga", "kolar", "bidar",
    "nellore", "tirupati", "anantapur", "kurnool", "nandyal", "guntur",
    "ongole", "eluru", "warangal", "karimnagar", "khammam", "nizamabad",
    "mahbubnagar", "srinagar", "leh", "kargil", "jammu", "ambala",
    "hisar", "rohtak", "panipat", "sonipat", "kurukshetra", "faridabad",
    "gurugram", "panchkula", "yamunanagar", "moradabad", "bareilly",
    "aligarh", "gorakhpur", "ghaziabad", "mathura", "firozabad", "saharanpur",
    "muzaffarnagar", "sitapur", "azamgarh", "etawah", "basti", "faizabad",
    "bhagalpur", "muzaffarpur", "gaya", "darbhanga", "katihar", "siwan",
    "motihari", "sasaram", "chhapra", "samastipur", "hajipur"
]

VALID_STATES = [
    "andhra pradesh", "arunachal pradesh", "assam", "bihar", "chhattisgarh", "goa",
    "gujarat", "haryana", "himachal pradesh", "jharkhand", "karnataka", "kerala",
    "madhya pradesh", "maharashtra", "manipur", "meghalaya", "mizoram", "nagaland",
    "odisha", "punjab", "rajasthan", "sikkim", "tamil nadu", "telangana", "tripura",
    "uttar pradesh", "uttarakhand", "west bengal", "delhi", "jammu and kashmir",
    "ladakh", "puducherry", "chandigarh", "dadra and nagar haveli and daman and diu",
    "lakshadweep", "andaman and nicobar islands"
]



CITY_ABBREV = {
    # Metro Abbreviations
    "mum": "mumbai", "bom": "mumbai",
    "pna": "pune", "pnq": "pune",
    "blr": "bengaluru", "bng": "bengaluru",
    "chn": "chennai", "maa": "chennai",
    "hyd": "hyderabad", "hydr": "hyderabad",
    "del": "delhi", "ndls": "delhi",
    "ggn": "gurgaon", "grg": "gurgaon", "ggrm": "gurugram",
    "kol": "kolkata", "ccu": "kolkata",
    "ahm": "ahmedabad", "amd": "ahmedabad",
    "srt": "surat", "stt": "surat",
    "lko": "lucknow", "lkd": "lucknow",
    "idr": "indore", "idrh": "indore",
    "bpl": "bhopal",
    "nash": "nashik", "nsk": "nashik",
    "ngp": "nagpur",
    "cbe": "coimbatore",
    "tvm": "trivandrum", "tvpm": "trivandrum",
    "vskp": "visakhapatnam", "vizag": "visakhapatnam",
    "vja": "vijayawada",
    "rpr": "raipur",
    "bhu": "bhubaneswar", "bbs": "bhubaneswar",
    "pat": "patna",
    "rnc": "ranchi",
    "ddn": "dehradun",
    "ptl": "patiala",
    "lko": "lucknow",
    "asr": "amritsar",
    "ldh": "ludhiana",
    # "ktm": "kochi",
    "mdr": "madurai",
    "mys": "mysore",
    "uda": "udaipur",
    "jpr": "jaipur",
    "jdh": "jodhpur",
    # "bdq": "vadodara",
    # "ixg": "belgaum",
    # "ixm": "madurai",
    # "tir": "tirupati",
    # "tcr": "thrissur",
    # "srr": "shoranur"
}

STATE_ABBREV = {
    # state abbreviations
    "ap": "andhra pradesh",
    "ar": "arunachal pradesh",
    "as": "assam",
    "br": "bihar",
    "cg": "chhattisgarh",
    # "ga": "goa",
    "gj": "gujarat",
    "hr": "haryana",
    "hp": "himachal pradesh",
    "jh": "jharkhand",
    "ka": "karnataka",
    "kl": "kerala",
    "mp": "madhya pradesh",
    "mh": "maharashtra",
    # "mn": "manipur",
    # "ml": "meghalaya",
    # "mz": "mizoram",
    # "nl": "nagaland",
    "od": "odisha",
    "pb": "punjab",
    "rj": "rajasthan",
    "sk": "sikkim",
    "tn": "tamil nadu",
    "ts": "telangana",
    # "tr": "tripura",
    "up": "uttar pradesh",
    "uk": "uttarakhand", "ut": "uttarakhand",
    "wb": "west bengal",
    "dl": "delhi",
    "jk": "jammu and kashmir",
    "ld": "ladakh",
    "py": "puducherry",
    "ch": "chandigarh",
    # "dn": "dadra and nagar haveli and daman and diu",
    # "dd": "daman and diu",
    # "la": "lakshadweep",
    # "an": "andaman and nicobar islands"
}

def clean_address(address):
    """Clean and preprocess address text"""
    address = re.sub(r"(https?://\S+)", "", address)
    address = re.sub(r"[\n\t\r]", " ", address)
    address = re.sub(r"[\'\"\`]s", "s", address)
    address = re.sub(r"[\'\"] ", " ", address)
    address = re.sub(r" [\'\"]", " ", address)
    address = re.sub(r"[a-zA-Z]{2,}[\'\"][a-zA-Z]{2,}", " ", address)
    address = re.sub(r"[\=\:\~]", "-", address)
    address = re.sub(r"[\[\{]", "(", address)
    address = re.sub(r"[\]\}]", ")", address)
    address = re.sub(r"[\|\\]", "/", address)
    address = re.sub(r"[;\?]", ",", address)
    address = re.sub(r"[`\!\$@\*%\<\>_\^]", " ", address)
    for ch in [",", ".", "+", "-", "(", ")", "&", "#", "/", "\"", "'"]:
        address = re.sub(f"\\{ch}+", ch, address)
    address = re.sub(" +", " ", address)
    address = address.strip()
    address = re.sub(r"^[\.\,\-\+\/\)]", "", address)
    address = re.sub(r"[\.\,\-\+\/\(]$", "", address)
    address = address.strip()
    parts = []
    for token in address.split():
        match_ = re.search("[\\\\.,;:\\-_]+$", token)
        if match_:
            token = re.sub("[\\\\.,;:\\-_]+$", " " + match_.group(0), token)
        parts.append(token)
    address = ' '.join(parts)
    address = address.lower()
    return address


ner_pipeline = pipeline(
    "ner",
    model="./address_ner_model_TinyBERT2",
    tokenizer="./address_ner_model_TinyBERT2",
    aggregation_strategy=None
)


def merge_subwords(preds):
    if not preds:
        return []
    merged = []
    current = preds[0].copy()
    for p in preds[1:]:
        if (p["entity"] == current["entity"] and p["start"] <= current["end"] + 1):
            current["end"] = p["end"]
            current["word"] += " " + p["word"].replace("##", "")
        else:
            merged.append(current)
            current = p.copy()
    merged.append(current)
    return merged


def normalize_entity(label, value, threshold=85):
    """Handle abbreviations and fuzzy normalization for city/state names."""
    if not value.strip():
        return None
    value = value.lower().strip()
    # Skip fuzzy match for short tokens (<= 2)
    if len(value) <= 2:
        return value

    # First, expand abbreviations
    if label == "city_name" and value in CITY_ABBREV:
        return CITY_ABBREV[value]
    elif label == "state_name" and value in STATE_ABBREV:
        return STATE_ABBREV[value]

    # Then fuzzy-match to handle misspellings
    if label == "city_name":
        best_match, score = process.extractOne(value, VALID_CITIES)
        if score >= threshold:
            return best_match
    elif label == "state_name":
        best_match, score = process.extractOne(value, VALID_STATES)
        if score >= threshold:
            return best_match

    return value


def get_entity_spans(text):
    preds = ner_pipeline(text)
    preds = merge_subwords(preds)

    entities = []
    for p in preds:
        start, end = p["start"], p["end"]
        label = p["entity"].split("-")[-1] if "-" in p["entity"] else p["entity"]
        entity_text = text[start:end].strip(" ,.")
        if not entity_text:
            continue

        token = entity_text.lower().strip()

        # Handle combined tokens like 'ka 560103' or 'tn600045'
        match = re.match(r"^([a-z]{2})\s*(\d{6})$", token)
        if match:
            state_code, pincode = match.groups()
            if state_code in STATE_ABBREV:
                entities.append([start, start + len(state_code), STATE_ABBREV[state_code], "state_name"])
            entities.append([end - 6, end, pincode, "area_pincode"])
            continue

        # Heuristic label correction
        if token in CITY_ABBREV:
            label = "city_name"
            entity_text = CITY_ABBREV[token]
        elif token in STATE_ABBREV:
            label = "state_name"
            entity_text = STATE_ABBREV[token]
        elif re.fullmatch(r"\b\d{6}\b", token):
            label = "area_pincode"

        # Apply normalization
        if label in ["city_name", "state_name"]:
            entity_text = normalize_entity(label, entity_text)
            if not entity_text:
                continue

        entities.append([start, end, entity_text, label])
    return entities



df = pd.read_csv("Assurekit_final_completeness.csv")
df=df.head(10)

annotated_data = []
for idx, row in df.iterrows():
    raw_text = str(row.get("customerAddress", "")).strip()
    if not raw_text or raw_text.lower() == "nan":
        continue
    text = clean_address(raw_text)
    try:
        entities = get_entity_spans(text)
        annotated_data.append({
            "text": text,
            "entities": entities
        })
    except Exception as e:
        print(f"⚠️ Error processing row {idx}: {e}")


output_file = "Assurekit_final_annotated.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(annotated_data, f, ensure_ascii=False, indent=4)

print(f"✅ Entity-level annotations (cleaned + abbrev-expanded + fuzzy-normalized) saved to {output_file}")
print(f"🧾 Total addresses processed: {len(annotated_data)}")


Device set to use mps:0
/var/folders/_3/j4fdj65x1hxdgmd_3jkytp980000gn/T/ipykernel_19769/1058128960.py:265: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Assurekit_final_completeness.csv")
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Entity-level annotations (cleaned + abbrev-expanded + fuzzy-normalized) saved to Assurekit_final_annotated.json
🧾 Total addresses processed: 10


Final Inference level logic

In [8]:
import re
import json
from transformers import pipeline
from fuzzywuzzy import process

VALID_CITIES = [
    # Tier 1 Cities
    "mumbai", "pune", "bengaluru", "bangalore", "chennai", "hyderabad", "delhi",
    "gurugram", "gurgaon", "noida", "kolkata", "ahmedabad", "surat", "jaipur",
    "lucknow", "indore", "bhopal", "nagpur", "chandigarh", "coimbatore", "visakhapatnam",
    "vijayawada", "patna", "ranchi", "kochi", "trivandrum", "madurai", "mysore",

    # Tier 2 / 3 Cities
    "amritsar", "jalandhar", "ludhiana", "varanasi", "kanpur", "allahabad",
    "meerut", "dehradun", "shimla", "haridwar", "patiala", "nashik", "aurangabad",
    "kolhapur", "rajkot", "vadodara", "gandhinagar", "bhavnagar", "jamnagar",
    "udaipur", "jodhpur", "ajmer", "bikaner", "alwar", "gwalior", "sagar",
    "rewa", "satna", "bilaspur", "raipur", "durg", "bhilai", "cuttack",
    "bhubaneswar", "puri", "rourkela", "silchar", "dibrugarh", "guwahati",
    "imphal", "aizawl", "shillong", "agartala", "itanagar", "gangtok",
    "panaji", "margao", "mapusa", "porvorim", "pondicherry", "karaikal",
    "tiruchirappalli", "salem", "vellore", "tirunelveli", "nagercoil",
    "eranakulam", "thrissur", "palakkad", "malappuram", "alappuzha",
    "hosur", "tumkur", "hubli", "dharwad", "belgaum", "mangalore",
    "bellary", "shimoga", "davangere", "chitradurga", "kolar", "bidar",
    "nellore", "tirupati", "anantapur", "kurnool", "nandyal", "guntur",
    "ongole", "eluru", "warangal", "karimnagar", "khammam", "nizamabad",
    "mahbubnagar", "srinagar", "leh", "kargil", "jammu", "ambala",
    "hisar", "rohtak", "panipat", "sonipat", "kurukshetra", "faridabad",
    "gurugram", "panchkula", "yamunanagar", "moradabad", "bareilly",
    "aligarh", "gorakhpur", "ghaziabad", "mathura", "firozabad", "saharanpur",
    "muzaffarnagar", "sitapur", "azamgarh", "etawah", "basti", "faizabad",
    "bhagalpur", "muzaffarpur", "gaya", "darbhanga", "katihar", "siwan",
    "motihari", "sasaram", "chhapra", "samastipur", "hajipur"
]

VALID_STATES = [
    "andhra pradesh", "arunachal pradesh", "assam", "bihar", "chhattisgarh", "goa",
    "gujarat", "haryana", "himachal pradesh", "jharkhand", "karnataka", "kerala",
    "madhya pradesh", "maharashtra", "manipur", "meghalaya", "mizoram", "nagaland",
    "odisha", "punjab", "rajasthan", "sikkim", "tamil nadu", "telangana", "tripura",
    "uttar pradesh", "uttarakhand", "west bengal", "delhi", "jammu and kashmir",
    "ladakh", "puducherry", "chandigarh", "dadra and nagar haveli and daman and diu",
    "lakshadweep", "andaman and nicobar islands"
]



CITY_ABBREV = {
    # Metro Abbreviations
    "mum": "mumbai", "bom": "mumbai",
    "pna": "pune", "pnq": "pune",
    "blr": "bengaluru", "bng": "bengaluru",
    "chn": "chennai", "maa": "chennai",
    "hyd": "hyderabad", "hydr": "hyderabad",
    "del": "delhi", "ndls": "delhi",
    "ggn": "gurgaon", "grg": "gurgaon", "ggrm": "gurugram",
    "kol": "kolkata", "ccu": "kolkata",
    "ahm": "ahmedabad", "amd": "ahmedabad",
    "srt": "surat", "stt": "surat",
    "lko": "lucknow", "lkd": "lucknow",
    "idr": "indore", "idrh": "indore",
    "bpl": "bhopal",
    "nash": "nashik", "nsk": "nashik",
    "ngp": "nagpur",
    "cbe": "coimbatore",
    "tvm": "trivandrum", "tvpm": "trivandrum",
    "vskp": "visakhapatnam", "vizag": "visakhapatnam",
    "vja": "vijayawada",
    "rpr": "raipur",
    "bhu": "bhubaneswar", "bbs": "bhubaneswar",
    "pat": "patna",
    "rnc": "ranchi",
    "ddn": "dehradun",
    "ptl": "patiala",
    "lko": "lucknow",
    "asr": "amritsar",
    "ldh": "ludhiana",
    # "ktm": "kochi",
    "mdr": "madurai",
    "mys": "mysore",
    "uda": "udaipur",
    "jpr": "jaipur",
    "jdh": "jodhpur",
    # "bdq": "vadodara",
    # "ixg": "belgaum",
    # "ixm": "madurai",
    # "tir": "tirupati",
    # "tcr": "thrissur",
    # "srr": "shoranur"
}

STATE_ABBREV = {
    # state abbreviations
    "ap": "andhra pradesh",
    "ar": "arunachal pradesh",
    "as": "assam",
    "br": "bihar",
    "cg": "chhattisgarh",
    # "ga": "goa",
    "gj": "gujarat",
    "hr": "haryana",
    "hp": "himachal pradesh",
    "jh": "jharkhand",
    "ka": "karnataka",
    "kl": "kerala",
    "mp": "madhya pradesh",
    "mh": "maharashtra",
    # "mn": "manipur",
    # "ml": "meghalaya",
    # "mz": "mizoram",
    # "nl": "nagaland",
    "od": "odisha",
    "pb": "punjab",
    "rj": "rajasthan",
    "sk": "sikkim",
    "tn": "tamil nadu",
    "ts": "telangana",
    # "tr": "tripura",
    "up": "uttar pradesh",
    "uk": "uttarakhand", "ut": "uttarakhand",
    "wb": "west bengal",
    "dl": "delhi",
    "jk": "jammu and kashmir",
    "ld": "ladakh",
    "py": "puducherry",
    "ch": "chandigarh",
    # "dn": "dadra and nagar haveli and daman and diu",
    # "dd": "daman and diu",
    # "la": "lakshadweep",
    # "an": "andaman and nicobar islands"
}


def clean_address(address: str) -> str:
    address = address.lower()
    address = re.sub(r"(https?://\S+)", "", address)
    address = re.sub(r"[\n\t\r]", " ", address)
    address = re.sub(r"[^\w\s,/-]", " ", address)
    address = re.sub(r",+", ",", address)
    address = re.sub(r"\s+", " ", address).strip()
    return address

def pre_normalize_abbrevs(text: str) -> str:
    words = text.split()
    new_words = []
    for w in words:
        if w in STATE_ABBREV:
            new_words.append(STATE_ABBREV[w])
        elif w in CITY_ABBREV:
            new_words.append(CITY_ABBREV[w])
        else:
            new_words.append(w)
    return " ".join(new_words)

ner_pipeline = pipeline(
    "ner",
    model="./address_ner_model_TinyBERT2",
    tokenizer="./address_ner_model_TinyBERT2",
    aggregation_strategy=None
)


def merge_subwords(preds):
    if not preds:
        return []
    merged = []
    current = preds[0].copy()
    for p in preds[1:]:
        if (p["entity"] == current["entity"] and p["start"] <= current["end"] + 1):
            current["end"] = p["end"]
            current["word"] += " " + p["word"].replace("##", "")
        else:
            merged.append(current)
            current = p.copy()
    merged.append(current)
    return merged

def normalize_entity(label, value, threshold=85):
    value = value.lower().strip()
    if not value:
        return None

    # Direct abbreviation match
    if label == "city_name" and value in CITY_ABBREV:
        return CITY_ABBREV[value]
    if label == "state_name" and value in STATE_ABBREV:
        return STATE_ABBREV[value]

    # Skip fuzzy match for short tokens (<= 2)
    if len(value) <= 2:
        return value

    # Fuzzy matching
    if label == "city_name":
        best_match, score = process.extractOne(value, VALID_CITIES)
        if score >= threshold:
            return best_match
    elif label == "state_name":
        best_match, score = process.extractOne(value, VALID_STATES)
        if score >= threshold:
            return best_match
    return value

def extract_address_entities(raw_address: str):
    text = clean_address(raw_address)
    text = pre_normalize_abbrevs(text)

    preds = ner_pipeline(text)
    preds = merge_subwords(preds)

    entities = []
    for p in preds:
        start, end = p["start"], p["end"]
        label = p["entity"].split("-")[-1] if "-" in p["entity"] else p["entity"]
        entity_text = text[start:end].strip(" ,.")
        if not entity_text:
            continue

        token = entity_text.lower().strip()

        # Handle combined tokens like 'ka560103'
        match = re.match(r"^([a-z]{2})\s*(\d{6})$", token)
        if match:
            state_code, pincode = match.groups()
            if state_code in STATE_ABBREV:
                entities.append([start, start + len(state_code), STATE_ABBREV[state_code], "state_name"])
            entities.append([end - 6, end, pincode, "area_pincode"])
            continue

        # Heuristic correction
        if token in CITY_ABBREV:
            label = "city_name"
            entity_text = CITY_ABBREV[token]
        elif token in STATE_ABBREV:
            label = "state_name"
            entity_text = STATE_ABBREV[token]
        elif re.fullmatch(r"\b\d{6}\b", token):
            label = "area_pincode"

        # Normalize
        if label in ["city_name", "state_name"]:
            entity_text = normalize_entity(label, entity_text)
            if not entity_text:
                continue

        entities.append([start, end, entity_text, label])

    return {"text": text, "entities": entities}

if __name__ == "__main__":
    address = "C-5 BLDG-11 FLAT NO 3 4TH FLOOR SECTOR-1A CBD BELAPUR NAVI MUMBAI MANGO GARDEN THANE 400614 MH,MH - 400614"
    result = extract_address_entities(address)
    print(json.dumps(result, indent=4, ensure_ascii=False))

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{
    "text": "c-5 bldg-11 flat no 3 4th floor sector-1a cbd belapur navi mumbai mango garden thane 400614 mh,mh - 400614",
    "entities": [
        [
            0,
            41,
            "c-5 bldg-11 flat no 3 4th floor sector-1a",
            "unit"
        ],
        [
            42,
            53,
            "cbd belapur",
            "area_name"
        ],
        [
            54,
            65,
            "mumbai",
            "city_name"
        ],
        [
            66,
            78,
            "mango garden",
            "area_name"
        ],
        [
            79,
            84,
            "thane",
            "city_name"
        ],
        [
            85,
            93,
            "400614 m",
            "area_pincode"
        ],
        [
            93,
            94,
            "h",
            "city_name"
        ],
        [
            95,
            97,
            "maharashtra",
            "state_name"
        ],
        [
           

Splitting the ner_address_corpus1 (renamed the Assurekit_final_annotated file) into train and test set

In [ ]:
import json
import random

input_file = "ner_address_corpus1.json"
train_file = "ner_list_train.json"
test_file = "ner_list_test.json"

with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

random.shuffle(data)

split_ratio = 0.8
split_index = int(len(data) * split_ratio)

train_data = data[:split_index]
test_data = data[split_index:]

with open(train_file, "w", encoding="utf-8") as f:
    json.dump(train_data, f, indent=4, ensure_ascii=False)

with open(test_file, "w", encoding="utf-8") as f:
    json.dump(test_data, f, indent=4, ensure_ascii=False)

print(f"Train set → {len(train_data)} samples saved to {train_file}")
print(f"Test set  → {len(test_data)} samples saved to {test_file}")


✅ Split complete:
Train set → 79464 samples saved to ner_list_train.json
Test set  → 19866 samples saved to ner_list_test.json
